In [ ]:
# Step1: Create & Setup hugging face API token in Collab

In [1]:
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-qiy32y5h
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-qiy32y5h
  Resolved https://github.com/unslothai/unsloth.git to commit 5bf1ce073ff6db8f4fd282b40686d8b4d780e942
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.5.4-py3-none-any.whl size=265196 sha256=9836104214ffde77ac6c27bbf89f6a72e010dea3a8788c8f492bbe0aadd84245
  Stored in directory: /tmp/pip-ephem-wheel-cache-176me3he/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.5.4
    Uninstalling unsloth-2025.5.4:
      Successfully uninstalled unsloth-2025.5.4


In [2]:
# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [4]:
# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [6]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.5.4: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
# Step6: Setup system prompt (Legal Version)
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a legal expert specializing in Indian Constitution, IPC, and CrPC. Answer the legal question below using your advanced knowledge.

### Query:
{}

### Answer:
{}
"""

In [19]:
# Step7: Run Inference on the model (Legal Version)

# Define a test legal question
question = """Under what circumstances can a person be granted anticipatory bail under CrPC?"""

# Prepare the model for inference
FastLanguageModel.for_inference(model)

# Tokenize the input using the original prompt style
inputs = tokenizer(
    [prompt_style.format(question, "")],
    return_tensors="pt"
).to("cuda")

# Generate a response (keeping original parameters)
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True
)

# Decode and print the full response
response = tokenizer.batch_decode(outputs)[0]
print(response)

<｜begin▁of▁sentence｜>
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a legal expert specializing in Indian Constitution, IPC, and CrPC. Answer the legal question below using your advanced knowledge.

### Query:
Under what circumstances can a person be granted anticipatory bail under CrPC?

### Answer:

**Step-by-Step Explanation:**

1. **Understanding the Term:**
   - Anticipatory bail is a type of bail granted by a court to an accused person before their arrest, ensuring they are released from custody.
   - It is typically sought when there is a fear of arrest due to a pending legal proceeding.

2. **Legal Basis:**
   - **Section 498 of the CrPC** deals with ant

In [22]:
print(response[0].split("### Answer")[1])


IndexError: list index out of range

In [29]:
!git clone https://huggingface.co/datasets/Techmaestro369/indian-legal-texts-finetuning

Cloning into 'indian-legal-texts-finetuning'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (13/13), 560.82 KiB | 2.53 MiB/s, done.


In [30]:
import json
from datasets import Dataset

def load_json_to_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return Dataset.from_list(data)

# Load each JSON file
constitution = load_json_to_dataset("indian-legal-texts-finetuning/constitution_qa.json")
ipc = load_json_to_dataset("indian-legal-texts-finetuning/ipc_qa.json")
crpc = load_json_to_dataset("indian-legal-texts-finetuning/crpc_qa.json")

In [31]:
import json
from datasets import Dataset

def load_json_to_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return Dataset.from_list(data)

# Load each JSON file
constitution = load_json_to_dataset("indian-legal-texts-finetuning/constitution_qa.json")
ipc = load_json_to_dataset("indian-legal-texts-finetuning/ipc_qa.json")
crpc = load_json_to_dataset("indian-legal-texts-finetuning/crpc_qa.json")

In [32]:
from datasets import concatenate_datasets

legal_dataset = concatenate_datasets([constitution, ipc, crpc])
legal_dataset = legal_dataset.shuffle(seed=42)  # Optional shuffling

In [33]:
print(legal_dataset)
print(legal_dataset[0])  # Check a sample

Dataset({
    features: ['question', 'answer'],
    num_rows: 14543
})
{'question': 'What does it mean when a Bill has been agreed to by both Houses, either without amendment or with such amendments only as are agreed to by both Houses?', 'answer': 'It means that the Bill is considered passed by the Houses of Parliament.'}


In [34]:

EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [35]:
### Finetuning (Legal Version)
# Updated training prompt style with legal focus
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a legal expert with advanced knowledge of Indian Constitution, IPC, and CrPC.
Please answer the following legal question.

### Question:
{}

### Legal Reasoning:
<think>
{}
</think>

### Response:
{}"""

In [36]:
# Prepare legal data for fine-tuning
def preprocess_input_data(examples):
    inputs = examples["question"]  # Changed from "Question" to match legal dataset
    outputs = examples["answer"]   # Changed from "Response"

    texts = []

    for input, output in zip(inputs, outputs):
        # Generate synthetic legal reasoning if Complex_CoT not available
        cot = f"""1. Jurisdiction: Identify relevant code (IPC/CrPC/Constitution)
2. Key Terms: {', '.join(input.split()[:3])}
3. Applicable Sections: [Relevant sections]
4. Precedent: [Landmark case if known]
5. Conclusion: {output[:50]}..."""  # First 50 chars of answer

        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}  # Changed key to singular "text" for SFTTrainer compatibility

In [37]:
# Corrected fine-tuning dataset preparation
finetune_dataset = legal_dataset.map(
    preprocess_input_data,
    batched=True,
    remove_columns=legal_dataset.column_names  # Clean up original columns
)

# Verify the processed dataset
print(finetune_dataset)
print(finetune_dataset[0]['text'])  # Inspect a sample

Map:   0%|          | 0/14543 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 14543
})
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a legal expert with advanced knowledge of Indian Constitution, IPC, and CrPC.
Please answer the following legal question.

### Question:
What does it mean when a Bill has been agreed to by both Houses, either without amendment or with such amendments only as are agreed to by both Houses?

### Legal Reasoning:
<think>
1. Jurisdiction: Identify relevant code (IPC/CrPC/Constitution)
2. Key Terms: What, does, it
3. Applicable Sections: [Relevant sections]
4. Precedent: [Landmark case if known]
5. Conclusion: It means that the Bill is considered passed by the...
</think>

### Response:
It means that the Bill is consi

In [38]:

# Step9: Setup/Apply LoRA finetuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.5.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [39]:
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate

In [40]:
# Corrected SFTTrainer Setup for Legal Fine-Tuning
trainer = SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=finetune_dataset,
    dataset_text_field="text",  # Changed from "texts" to match your preprocessed data
    max_seq_length=max_sequence_length,
    dataset_num_proc=2,  # Increased from 1 for faster preprocessing
    packing=True,  # Enable for more efficient training

    # Optimized TrainingArguments
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # Better than linear for legal tasks
        seed=3407,
        output_dir="legal_outputs",  # Changed from generic "outputs"
        report_to="wandb",  # Enable if using Weights & Biases
        save_strategy="steps",
        save_steps=20
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/14543 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [41]:
# Setup W&B for Legal Fine-Tuning
from google.colab import userdata
import wandb

# 1. Secure Login
wnb_token = userdata.get("WANDB_API_TOKEN")  # Retrieve from Colab secrets
wandb.login(key=wnb_token)

# 2. Initialize Run with Legal-Specific Config
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Indian-Legal-Texts',  # Updated project name
    job_type="legal_finetuning",  # More specific job type
    config={
        "model": "DeepSeek-R1-Llama-8B",
        "task": "Indian Legal Q&A",
        "dataset": "IPC/CrPC/Constitution",
        "peft": "LoRA",
        "batch_size": 2,
        "lr": 2e-4
    }
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mdumarshaikh (mdumarshaikh-trinity-academy-of-engineering-pune) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [42]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,543 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.281200
20,0.586900
30,0.482300
40,0.419400
50,0.439500
60,0.449900


In [43]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▂▁▂▁
train/learning_rate,█▇▅▃▂▁
train/loss,█▂▁▁▁▁
total_flos,4797000430043136.0
train/epoch,0.033
train/global_step,60
train/grad_norm,0.34037
train/learning_rate,0.0
train/loss,0.4499


In [44]:
# Step10: Testing after fine-tuning (Legal Version)
question = """Under what circumstances can anticipatory bail be granted under CrPC?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize using the same prompt style
inputs = tokenizer(
    [prompt_style.format(question, "")],  # Unchanged prompt structure
    return_tensors="pt"
).to("cuda")

# Generate response (same parameters)
outputs = model_lora.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=400,  # Reduced from 1200 for legal answers
    use_cache=True
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response.split("### Answer:")[1].strip())  # Same extraction method

Under the circumstances specified in sub-section (1) of section 498, anticipatory bail may be granted by the Court.<｜end▁of▁sentence｜>
